#Text to Speech Fine Tuning

In [ ]:
!nvidia-smi

Sat Aug 30 07:15:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Data Genration & Preparation

Data : Upload Recording -> Whisper -> HF Dataset(Hugging Face Dataset)

In [ ]:
# Installing Dependencies
!pip -q install --upgrade ffmpeg-python \
                     git+https://github.com/openai/whisper.git \
                     soundfile datasets

# Imports
import os, json, subprocess, uuid, torch, whisper, glob
from pathlib import Path

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
print("CUDA available:", torch.cuda.is_available())  # For checking if the Model_size & Device is available as mentioned

CUDA available: True


In [ ]:
# Whisper Model & Hardware
MODEL_SIZE = "turbo"   # tiny | base | small | medium | large-v3 | turbo
DEVICE = "cuda"

# Where all audio files will live
AUDIO_DIR = Path("/content/audio")
AUDIO_DIR.mkdir(exist_ok=True, parents=True)

print(f" Using {MODEL_SIZE} model on {DEVICE}")

 Using turbo model on cuda


# Upload Audio File (.mp3 or .wav or .m4a)

In [ ]:
from google.colab import files
uploaded = files.upload() # Helps in picking Multiple files

for name, _ in uploaded.items():
  dest = AUDIO_DIR / name
  os.rename(name, dest)  # Will move into AUDIO_DIR
  print("Saved", dest)
  print("All uploads saved to", AUDIO_DIR)


Saving Voice_Cloning_Audio_Raw.mp3 to Voice_Cloning_Audio_Raw.mp3
Saved /content/audio/Voice_Cloning_Audio_Raw.mp3
All uploads saved to /content/audio


## Gather Audio Files, Clean & Transcribe

In [ ]:
from pathlib import Path
import subprocess, json

def get_duration_sec(path: Path) -> float:
    pr = subprocess.run(
        ["ffprobe", "-v", "error",
         "-show_entries", "format=duration",
         "-of", "json", str(path)],
        capture_output=True, text=True
    )
    try:
        return float(json.loads(pr.stdout)["format"]["duration"])
    except (json.JSONDecodeError, KeyError, ValueError) as e:
        print(f"Error getting duration for {path}: {e}")
        return 0.0


uploaded_file = Path("audio/Voice_Cloning_Audio_Raw.mp3")
duration = get_duration_sec(uploaded_file)
print(f"Duration of {uploaded_file.name}: {duration/60:.2f} min ({duration/3600:.2f} h)")


Duration of Voice_Cloning_Audio_Raw.mp3: 29.18 min (0.49 h)


In [ ]:
from pathlib import Path
import subprocess

# Creating output directory
CLEANED_AUDIO_DIR = Path("/content/audio_clean")
CLEANED_AUDIO_DIR.mkdir(exist_ok=True)

# Defining the cleaning function
def clean_one(src: Path) -> Path:
    out = CLEANED_AUDIO_DIR / f"{src.stem}_clean.wav"
    filt = (
        "highpass=f=80,"
        "afftdn,"
        "loudnorm=I=-16:LRA=11:TP=-1.5,"
        "dynaudnorm=f=200,"
        "apad=pad_dur=0.1"
    )
    subprocess.run([
        "ffmpeg", "-y",
        "-i", str(src),
        "-af", filt,
        "-ar", "24000",
        "-ac", "1",
        str(out)
         ], check=True)
    return out
uploaded_file = Path("audio/Voice_Cloning_Audio_Raw.mp3")
cleaned_file = clean_one(uploaded_file)

print(f"Cleaned audio saved to: {cleaned_file}")

Cleaned audio saved to: /content/audio_clean/Voice_Cloning_Audio_Raw_clean.wav


In [ ]:
# Converting My original file format from .mp3 to .wav
from pydub import AudioSegment

# Load the MP3 file
mp3_audio = AudioSegment.from_mp3("audio/Voice_Cloning_Audio_Raw.mp3")

# Export as WAV
mp3_audio.export("Voice_Cloning_Audio_Raw.wav", format="wav")


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


<_io.BufferedRandom name='Voice_Cloning_Audio_Raw.wav'>

In [ ]:
from pathlib import Path
from IPython.display import Audio
import subprocess

# Use your uploaded audio file directly
print("Original Audio File (first 10 sec)")
uploaded_audio_path = Path("Voice_Cloning_Audio_Raw.wav")

# Extract the first 10 seconds
temp_original_clip_path = Path("temp_original_clip_10s.wav")
subprocess.run([
    "ffmpeg", "-y", "-loglevel", "error",
    "-i", str(uploaded_audio_path),
    "-ss", "0", "-t", "10",
    str(temp_original_clip_path)
], check=True)

# Play the extracted clip
display(Audio(str(temp_original_clip_path)))

# Get duration
original_clip_duration = get_duration_sec(temp_original_clip_path)
print(f"Duration: {original_clip_duration:.3f} seconds")

# Clean the clip
print("\nCleaning the first 10 seconds...")
temp_cleaned_clip_path = clean_one(temp_original_clip_path)

# Get cleaned duration
cleaned_clip_duration = get_duration_sec(temp_cleaned_clip_path)
print(f"Duration: {cleaned_clip_duration:.3f} seconds")

display(Audio(str(temp_cleaned_clip_path)))
# Optional: Remove temporary files
temp_original_clip_path.unlink()
temp_cleaned_clip_path.unlink()

Original Audio File (first 10 sec)


Duration: 10.000 seconds

Cleaning the first 10 seconds...
Duration: 10.070 seconds


## Transcribe the cleaned file

In [ ]:
from pathlib import Path
import whisper
import json

# Load Whisper model
model = whisper.load_model(MODEL_SIZE, device=DEVICE)

# Define cleaned file (already created earlier)
cleaned_file = Path("audio_clean/Voice_Cloning_Audio_Raw_clean.wav")

# Create transcripts directory
TRANSCRIPTS_DIR = Path("/transcripts")
TRANSCRIPTS_DIR.mkdir(exist_ok=True)

# Prepare output path
fname = cleaned_file.stem
out_json = TRANSCRIPTS_DIR / f"{fname}_whisper.json"

# Transcribe if not already done
if out_json.exists():
    print(f" Skipping transcription for {fname} (already exists)")
else:
    print(f" Transcribing {fname} ...")

    result = model.transcribe(
        str(cleaned_file),
        fp16=False,
        verbose=False,
        word_timestamps=True
    )
    with out_json.open("w") as f:
        json.dump(result, f, indent=2)

print(f"Done! Transcript saved to:", out_json)



 Skipping transcription for Voice_Cloning_Audio_Raw_clean (already exists)
Done! Transcript saved to: /transcripts/Voice_Cloning_Audio_Raw_clean_whisper.json


## Chunk The audio, respecting sentences

In [ ]:
from huggingface_hub import login, HfApi
from huggingface_hub.utils import LocalTokenNotFoundError

# Check if already logged in
try:
    api = HfApi()
    api.whoami()
    print("Already logged in")
except LocalTokenNotFoundError:
    print("Not logged into Hugging Face. Logging in...")
    login()


Already logged in


In [ ]:
from pathlib import Path
import json, subprocess, uuid, datasets, os


REPO_NAME = "vaishnawi_tts_cleaned_audio"  # Replace with your repo name
MAX_SEC = 30  # Duration limit for audio chunks

# Required for CSM 1-b
sampling_rate = 24_000


In [ ]:
audio_path = Path("/content/audio_clean/Voice_Cloning_Audio_Raw_clean.wav")
json_path = Path("/transcripts/Voice_Cloning_Audio_Raw_clean_whisper.json")
CLIPS_ROOT = Path("/content/Clips")
CLIPS_ROOT.mkdir(exist_ok=True)

In [ ]:
from pathlib import Path
import json, subprocess, uuid, os, datasets
import nltk
from nltk.tokenize import PunktSentenceTokenizer

# Make sure punkt is available
nltk.download("punkt", quiet=True)

# Directories
TRANSCRIPTS_DIR = Path("/transcripts")
CLIPS_ROOT = Path("/content/Clips")
CLIPS_ROOT.mkdir(exist_ok=True)
CLEANED_AUDIO_DIR = Path("/content/audio_clean")  # where your cleaned audio is


# ------------------- CHUNKING FUNCTION -------------------
def chunk_one(audio_path: Path, json_path: Path, MAX_SEC=30):
    """
    Returns list of dict: {audio, text, source} for one file.
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    segments = data["segments"]

    # ---------------- build word-level metadata ----------------
    word_meta, full_text, char_pos = [], [], 0
    for seg in segments:
        if "words" in seg:  #  use word-level timestamps
            for w in seg["words"]:
                tok = w["word"].strip()
                if not tok:
                    continue
                start_char, end_char = char_pos, char_pos + len(tok)
                word_meta.append({
                    "char0": start_char,
                    "char1": end_char,
                    "t0": w["start"],
                    "t1": w["end"]
                })
                full_text.append(tok)
                char_pos = end_char + 1   # +1 for space
        else:  #  fallback to segment-level
            tok = seg["text"].strip()
            if not tok:
                continue
            start_char, end_char = char_pos, char_pos + len(tok)
            word_meta.append({
                "char0": start_char,
                "char1": end_char,
                "t0": seg["start"],
                "t1": seg["end"]
            })
            full_text.append(tok)
            char_pos = end_char + 1

    # join all words into one string
    full_text = " ".join(full_text)

    # ---------------- sentence spans with NLTK ----------------
    tokenizer = PunktSentenceTokenizer()
    sent_spans = list(tokenizer.span_tokenize(full_text))

    sentences, w_idx = [], 0
    for c0, c1 in sent_spans:
        # map chars → word indices → times
        first_w = w_idx
        while first_w < len(word_meta) and word_meta[first_w]["char0"] <= c0:
            first_w += 1
        last_w = first_w
        while last_w < len(word_meta) and word_meta[last_w]["char1"] < c1:
            last_w += 1
        if first_w == last_w:  # safeguard
            continue

        s_time = word_meta[first_w]["t0"]
        e_time = word_meta[last_w - 1]["t1"]
        text = full_text[c0:c1]

        sentences.append({"start": s_time, "end": e_time, "text": text})
        w_idx = last_w

    # ---------------- bundle sentences ≤ MAX_SEC ----------------
    clip_rows, bundle = [], None

    def flush(b):
        if b is None:
            return
        st, et, tx, out_dir = b
        out_dir.mkdir(exist_ok=True, parents=True)
        clip_path = out_dir / f"clip_{uuid.uuid4().hex}.wav"
        subprocess.run([
            "ffmpeg", "-loglevel", "error", "-y",
            "-i", str(audio_path),
            "-ss", str(st), "-to", str(et),
            "-ar", "16000", "-ac", "1",
            str(clip_path)
        ], check=True)
        clip_rows.append({
            "audio": str(clip_path),
            "text": tx.strip(),
            "source": str(audio_path)
        })

    out_dir = CLIPS_ROOT / audio_path.stem
    for s in sentences:
        st, et, tx = s["start"], s["end"], s["text"]
        dur = et - st
        if dur > MAX_SEC:
            continue  # skip too long
        if bundle is None:
            bundle = (st, et, tx, out_dir)
            continue
        b_st, b_et, b_tx, _ = bundle
        if (et - b_st) <= MAX_SEC:
            bundle = (b_st, et, b_tx + " " + tx, out_dir)
        else:
            flush(bundle)
            bundle = (st, et, tx, out_dir)

    flush(bundle)
    return clip_rows


In [ ]:
audio_path = Path("/content/audio_clean/Voice_Cloning_Audio_Raw_clean.wav")
json_path = Path("/transcripts/Voice_Cloning_Audio_Raw_clean_whisper.json")

clip_rows = chunk_one(audio_path, json_path)

print(f" Created {len(clip_rows)} clips")


✅ Created 29 clips


In [ ]:
from datasets import Dataset, Audio
import pandas as pd
from pathlib import Path

# --- Build dataframe from your clips ---
clips_root = Path("/content/Clips")
rows = []
for subdir in clips_root.iterdir():
    if subdir.is_dir():
        for wav in subdir.glob("*.wav"):
            # assuming you stored transcripts alongside
            # here we'll just fake by using filename -> you can adjust
            text_file = wav.with_suffix(".txt")
            if text_file.exists():
                text = text_file.read_text().strip()
            else:
                text = ""  # fallback if no transcript file
            rows.append({"audio": str(wav), "text": text, "source": subdir.name})

df = pd.DataFrame(rows)
print("Total rows:", len(df))
print(df.head())

# --- Convert to HF Dataset ---
ds = Dataset.from_pandas(df)

# Cast audio column (⚠ no decoding triggered when just uploading)
ds = ds.cast_column("audio", Audio(sampling_rate=24000, decode=False))

# --- Push to HF Hub ---
HF_REPO = "VAISHNAWI1/vaishnawi_tts_cleaned_audio"

ds.push_to_hub(HF_REPO)


Total rows: 58
                                               audio text  \
0  /content/Clips/Voice_Cloning_Audio_Raw_clean/c...        
1  /content/Clips/Voice_Cloning_Audio_Raw_clean/c...        
2  /content/Clips/Voice_Cloning_Audio_Raw_clean/c...        
3  /content/Clips/Voice_Cloning_Audio_Raw_clean/c...        
4  /content/Clips/Voice_Cloning_Audio_Raw_clean/c...        

                          source  
0  Voice_Cloning_Audio_Raw_clean  
1  Voice_Cloning_Audio_Raw_clean  
2  Voice_Cloning_Audio_Raw_clean  
3  Voice_Cloning_Audio_Raw_clean  
4  Voice_Cloning_Audio_Raw_clean  


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Map:   0%|          | 0/58 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :   4%|3         |  542kB / 14.5MB            

CommitInfo(commit_url='https://huggingface.co/datasets/VAISHNAWI1/vaishnawi_tts_cleaned_audio/commit/c2c38a95b587ae039a211c40750457f1841ee04c', commit_message='Upload dataset', commit_description='', oid='c2c38a95b587ae039a211c40750457f1841ee04c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/VAISHNAWI1/vaishnawi_tts_cleaned_audio', endpoint='https://huggingface.co', repo_type='dataset', repo_id='VAISHNAWI1/vaishnawi_tts_cleaned_audio'), pr_revision=None, pr_num=None)

## Unsloth


In [ ]:
%%capture
import os
if "COLAB" not in "".join(os.environ.keys()):
    !pip install unsloth tensorboard
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo tensorboard
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.52.3


In [ ]:
from unsloth import FastModel
from transformers import CsmForConditionalGeneration
import torch

model_name = "unsloth/csm-1b"
# model_name = "Trelis/csm-voice"

model, processor = FastModel.from_pretrained(
    model_name = model_name,
    max_seq_length= 2048,
    dtype = None, # Leave as None for auto-detection (will be torch.float16 on T4, torch.bfloat16 on h100 or blackwell GPUs)
    auto_model= CsmForConditionalGeneration, # Model configuration,
    load_in_4bit = False, # Set True for 4bit - reduces memory usage
    full_finetuning = True # Comment out if using LoRA
)


==((====))==  Unsloth 2025.8.10: Fast Mimi patching. Transformers: 4.52.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Float16 full finetuning uses more memory since we upcast weights to float32.


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

unsloth/csm-1b does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


In [ ]:
print(model)

CsmForConditionalGeneration(
  (lm_head): Linear(in_features=2048, out_features=2051, bias=False)
  (embed_text_tokens): Embedding(128256, 2048)
  (backbone_model): CsmBackboneModel(
    (embed_tokens): CsmBackboneModelEmbeddings(
      (embed_audio_tokens): Embedding(65632, 2048)
    )
    (layers): ModuleList(
      (0-15): 16 x CsmDecoderLayer(
        (self_attn): CsmAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): CsmMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        

## Data Prep

In [ ]:
ft_dataset = "VAISHNAWI1/vaishnawi_tts_cleaned_audio"
sampling_rate = 24_000

## Dataset Prep Functions

In [ ]:
# Dataset Prep functions
from datasets import load_dataset, Audio, Dataset
import os
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("unsloth/csm-1b")

raw_ds = load_dataset(ft_dataset, split="train")
print(f"Dataset loaded with features: {raw_ds.features["audio"]}")

# Ensure there is a column identifying the speaker for each row
# Getting the speaker id is important for multi-speaker models and speaker consistency
speaker_key = "source"
if "source" not in raw_ds.column_names:
    print("Unsloth: Not speaker found, adding default \"source\" of 0 for all examples")
    nrw_column = ["0"] * len(raw_ds)
    raw_ds = raw_ds.add_column("source",new_column)
elif "source" not in raw_ds.column_names:
    speaker_key = "source"

target_sampling_rate = sampling_rate
raw_ds = raw_ds.cast_column("audio", Audio(sampling_rate=target_sampling_rate))

# split into train / eval
import math
from datasets import DatasetDict

total_rows = len(raw_ds)
eval_rows = min(30, max(1, math.ceil(0.10 * total_rows))) #s30 or 10 %
# Hugging Face 'train_test_split' accepts an int for exact row counts
split: DatasetDict = raw_ds.train_test_split(
    test_size = eval_rows,
    shuffle = True,
    seed =42,   # make the split reproducable
)

raw_train_ds, raw_eval_ds = split["train"], split["test"]

print(f"Train Split: {len(raw_train_ds):>5} rows")
print(f"Eval Split: {len(raw_eval_ds):>5} rows (requested {eval_rows})")


Dataset loaded with features: Audio(sampling_rate=24000, decode=False, stream_index=None)
Train Split:    52 rows
Eval Split:     6 rows (requested 6)


In [ ]:
!pip install torchcodec -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
import torchcodec
print(torchcodec.__version__)


0.6.0


In [ ]:
!pip install --upgrade torch torchvision torchaudio


In [ ]:
!pip install --upgrade --force-reinstall torchcodec


  Using cached torchcodec-0.6.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (10 kB)
Using cached torchcodec-0.6.0-cp312-cp312-manylinux_2_28_x86_64.whl (1.4 MB)
  Attempting uninstall: torchcodec
    Found existing installation: torchcodec 0.6.0
    Uninstalling torchcodec-0.6.0:
      Successfully uninstalled torchcodec-0.6.0


In [ ]:
# Print out the longest text length
max_text_length = 0
for example in raw_ds:
    # Access the length of the text string
    text_length = len(example["text"])
    if text_length > max_text_length:
        max_text_length = text_length

print(f"Maximum text length in the dataset: {max_text_length}")

max_audio_length = 0
for example in raw_ds:
    try:
        audio_array = example["audio"]["array"]
        audio_length = len(audio_array)
        if audio_length > max_audio_length:
            max_audio_length = audio_length
    except Exception as e:
        print("Skipping file due to error:", e)

print(f"Maximum audio length in the dataset: {max_audio_length}")


Maximum text length in the dataset: 0
Skipping file due to error: The frame has 0 channels, expected 1. If you are hitting this, it may be because you are using a buggy FFmpeg version. FFmpeg4 is known to fail here in some valid scenarios. Try to upgrade FFmpeg?
Skipping file due to error: The frame has 0 channels, expected 1. If you are hitting this, it may be because you are using a buggy FFmpeg version. FFmpeg4 is known to fail here in some valid scenarios. Try to upgrade FFmpeg?
Skipping file due to error: The frame has 0 channels, expected 1. If you are hitting this, it may be because you are using a buggy FFmpeg version. FFmpeg4 is known to fail here in some valid scenarios. Try to upgrade FFmpeg?
Skipping file due to error: The frame has 0 channels, expected 1. If you are hitting this, it may be because you are using a buggy FFmpeg version. FFmpeg4 is known to fail here in some valid scenarios. Try to upgrade FFmpeg?
Skipping file due to error: The frame has 0 channels, expected

In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y ffmpeg
!ffmpeg -version


Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,20